In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


[<img src="https://www.digitalhouse.com/ar/logo-DH.png" width="400" height="200" align='right'>](http://digitalhouse.com.ar/)

# Regresión Lineal con scikit-learn y statsmodels

## Resumen
<div id="caja4" style="float:left;width: 100%;">
  <div style="float:right;width: 15%;"><img src="../../../common/icons/en_resumen.png" style="align:right"/> </div>
  <div style="float:right;width: 85%;"><label></label></div>
</div>



Hoy vamos a investigar un dataset clásico en el mundo de data science (DS), el dataset de propiedades de la ciudad de  Boston. 

En particular aplicaremos la regresion lineal a un problema inmobiliario. Sin embargo, descubriremos que esta técnica puede ser muy útil en otras areas como: marketing, finanzas, biología, petroquímica, etc. A lo largo de esta notebook vamos a usar dos librerías (o paquetes) diferentes, para los que pueden ver ejemplos en la documentación oficial.

* statsmodels -- [docs regresión lineal](http://statsmodels.sourceforge.net/devel/examples/#regression)
* scikit-learn -- [docs regresión lineal](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

## 1. Introducción

Ya nos hemos familizarizado bastante con scikit-learn, e iremos viendo la potencia de la misma. Sin embargo también debemos saber que  la librería incluye algunos datasets para testear y practicar los algoritmos que tiene implementado. El listado de los datasets que podemos encontrar los podes ver [aquí](https://scikit-learn.org/stable/datasets/index.html). 

Para cargar el dataset con el que vamos a trabajar hoy simplemente hay que importarlo desde sklearn.datasets

In [1]:
# Importamos algunas librerias para graficar.
%matplotlib inline
from matplotlib import pyplot as plt


In [2]:
# importamos el modelo lineal y algunas funciones para calcular la bondad de ajuste.
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

## 2. Conociendo el formato de los datsets de scikit-learn

In [3]:
# Para cargar el dataset lo importamos desde scikit-learn.
from sklearn import datasets
data = datasets.load_boston()

c:\Users\Ionatan\anaconda3\envs\DH\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing 

In [4]:
# imprimimos la ayuda para ver todos los datasets dispoibles.
datasets??

Type:        module
String form: <module 'sklearn.datasets' from 'c:\\Users\\Ionatan\\anaconda3\\envs\\DH\\lib\\site-packages\\sklearn\\datasets\\__init__.py'>
File:        c:\users\ionatan\anaconda3\envs\dh\lib\site-packages\sklearn\datasets\__init__.py
Source:     
"""
The :mod:`sklearn.datasets` module includes utilities to load datasets,
including methods to load and fetch popular reference datasets. It also
features some artificial data generators.
"""
from ._base import load_breast_cancer
from ._base import load_boston
from ._base import load_diabetes
from ._base import load_digits
from ._base import load_files
from ._base import load_iris
from ._base import load_linnerud
from ._base import load_sample_images
from ._base import load_sample_image
from ._base import load_wine
from ._base import get_data_home
from ._base import clear_data_home
from ._covtype import fetch_covtype
from ._kddcup99 import fetch_kddcup99
from ._lfw import fetch_lfw_pairs
from ._lfw import fetch_lfw_peopl

Para echarles un vistazo podemos ver con el atributo DESCR la descripcion del dataset. 

In [ ]:
print (data.DESCR)

El dataset lista 14 atributos que describen hogares de boston. Como dice la descripción el valor medio es el atributo que habitualmente se utiliza como variable objetivo. En nuestro caso mas sencillo es el atributo que intentaremos predecir a partir de otras variables explicativas. Cada instancia del dataframe describe un área suburbana o municipio de Boston ( Boston Standard Metropolitan Statistical Area (SMSA)) en 1970.

In [ ]:
type(data)

> Un Bunch es un diccionario de Python dictionary que provee acceso utilizando atributos. Un Bunch es una subclase de dict y soporta todos los métodos de un dict.

In [ ]:
#Dentro del Bunch podemos obtener diferentes atributos de nuestro dataset.
# Los nombres de las columnas 
print (data.feature_names)
# Los predictores o variables independientes.
print (data.data[0])
# La variable objetivo o variable dependiente.
print (data.target[0])

Como pueden ver, Scikit-learn ya separó el precio medio  de las casas del resto de las variables. El atributo target u objetivo (data.target) es nuestra variable dependiente intentaremos predecir a partir de los demás.

## 3. Estimando el modelo de regresión lineal

Primero, pongamos los datos en un data frame y asegurémonos de que esté todo cargado correctamente

In [5]:
import numpy as np
import pandas as pd
df = pd.DataFrame(data.data, columns=data.feature_names)

# Ponemos el target (precio de las casas -- MEDV) en otro DataFrame
targets = pd.DataFrame(data.target, columns=["MEDV"])

# Miremos las primeras filas de datos
display (df.head())
display (targets.head())

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


,MEDV
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


El primer ejercicio podria ser: Seleccionar variables una por una y ver su relación con la variable objetivo.

* Identifiquemos visualmente algunas variables que parezcan relacionadas al precio. Por ejemplo: RM and LSTAT. 

* Hagamos un análisis por separado y luego juntas.

## Relación entre RM y MEDV

 Generamos las matrices de variables explicativas o independientes y el target.
 
 Primero seleccionamos como variable explicativa a RM como numero promedio de habitaciones.
 
 
 ### Para Pensar...
 
 <div id="caja9" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/para_seguir_pensando.png" style="align:left"/> </div>
  <div style="float:left;width: 85%;"><label>Tiene sentido que exista una relacion lineal entre el numero promedio de habitaciones y el precio promedio?.
</label></div>
</div>
 

In [6]:
# Seleccionamos la variable predictora y la objetivo.
X = df[["RM"]]
y = targets["MEDV"]

# Importamos, Instanciamos, Fiteamos, etc..

# Instanciamos el modelo.
lm = linear_model.LinearRegression()

# Fiteamos el modelo sobre los vectores X e y.
model = lm.fit(X, y)
#
# Guardamos  las predicciones en un nuevo vector que llamaremos predictions.
predictions = lm.predict(X)

# Imprimimos el intercepto y los coeficientes como atributos del objeto entrenado.
print ('Intercepto=', ' ', model.intercept_)
print ('RM=', ' ', model.coef_)
# imprimos la metrica que mide la bondad de ajusto del modelo. En este caso el R2.
print ('R2_train=', ' ', model.score(X, y))

Intercepto=   -34.670620776438554
RM=   [9.10210898]
R2_train=   0.48352545599133423


In [ ]:
# Generamos una función que resume los coeficientes, el intercepto y el R2
# "model" = objeto con el modelo
# "X" = matrix de variables independientes

def sum_mod(model, X):
    a = pd.DataFrame(model.coef_ , X.columns.values)
    a = a.append(pd.DataFrame([model.intercept_, model.score(X, y)], index=['Intecept','R2']))
    return(a)

In [ ]:
# Graficamos la variable X contra la variable Y
plt.scatter(X, y, s=30, c='black', marker='+', zorder=10)
plt.scatter(X, y)
plt.xlabel("RM")
plt.ylabel("Valores reales MEDV")
plt.title('Relación entre RM y MEDV')
plt.show()

# Graficamos el modelo
plt.plot(y,y, '-.',c='grey')
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicciones de MEDV usando RM")
plt.ylabel("Valores reales MEDV")
plt.title('Comparación entre el modelo y los valores reales de MEDV')
plt.show()

In [ ]:
print ("EMC:", mean_squared_error(y, predictions)) # error medio cuadrático
sum_mod(model, X)

* ¿Qué pueden decir al comparar los dos gráficos? ¿Cómo interpretan el último gráfico ? ¿Qué les dice acerca del modelo? ¿Qué pueden decir acerca de la relación entre RM y MEDV? 

* Repitamos ahora lo anterior pero usando otra variable...


El primer gráfico tiene como objetivo evaluar si existe algún tipo de relacion lineal entre la variable explicativa y la variable objetivo. En particular con estas variables se observa que hay cierta correlación entre ellas, puesto que cuando el número de Habitaciones promedio aumenta, el precio medio de la zona también aumenta, lo cual tiene cierto sentido. 
 
 El segundo gráfico es la comparación entre el **valor real observado** vs el **valor predicho** por nuestro modelo. Es decir obtuvimos cuál es la relación entre la variable RM y MEDV a través de un vector de coeficientes y un intercepto.
 
 Como se interpreta el **EMC** porque usamos esa metrica? existe alguna mejor que otra? Que es el R2, puede dar negativo?. 
 
 Algunos recursos para repasar estos [conceptos](https://towardsdatascience.com/regression-an-explanation-of-regression-metrics-and-what-can-go-wrong-a39a9793d914)

## Relación entre LSTAT y MEDV

De manera muy similar al ejercicio anterior, observamos si existe relación entre la variable LSTAT (porcentaje con menos status o menor poder adquisitivo de la población) y el precio promedio (MEDV). Esta variable (LSTAT) podría tener alguna relación inversa con la variable objetivo (precios), puesto que podemos inferir que mientras mayor es el porcentaje de la población que tiene menor poder adquisitivo, más baratas (en promedio) deberían ser las casas. Por lo que esperamos una relación inversa entre el índice LSTAT y la variable objetivo. 

In [ ]:
# Instanciamos el nuevo modelo.
lm = linear_model.LinearRegression()

#  Seleccionamos la variable a testear.
X = df[["LSTAT"]]
y = targets["MEDV"]

# Fiteamos y realizamos las predicciones.
model = lm.fit(X, y)
predictions = lm.predict(X)

# Graficamos la variable X contra la variable Y
plt.scatter(X, y, s=30, c='black', marker='+', zorder=10)
plt.xlabel("LSTAT")
plt.ylabel("Valores reales MEDV")
plt.title('Relación entre LSTAT y MEDV')
plt.show()

# Graficamos el modelo
plt.plot(y,y, '-.',c='grey')
plt.scatter(predictions, y, s=30, c='r', marker='o', zorder=10)
plt.xlabel("Predicciones de MEDV usando LSTAT")
plt.ylabel("Valores reales MEDV")
plt.title('Comparación entre el modelo y los valores reales de MEDV')
plt.show()
print ("EMC:", mean_squared_error(y, predictions))
sum_mod(model, X)

* ¿Qué pueden decir al comparar éste modelo y el anterior? 

Como anticipamos el coeficiente beta de la variable LSTAT es negativo. Cuál es la interpretación del intercepto?.

* Estimemos, ahora, un modelo usando las dos variables anteriores.

## Model lineal con dos variables. LSTAT y RM

In [ ]:
# Instancio el modelo.
lm = linear_model.LinearRegression()

X = df[["RM", "LSTAT"]]
y = targets["MEDV"]

# Fiteo el modelo y hago las predicciones.
model = lm.fit(X, y)
predictions = model.predict(X)

# Graficamos el modelo
plt.plot(y,y, '-.',c='grey')
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicciones de MEDV usando RM y LSTAT")
plt.ylabel("Valores reales MEDV")
plt.show()
print ("EMC:", mean_squared_error(y, predictions))
prevMSE = mean_squared_error(y, predictions)

In [ ]:
sum_mod(model, X)

Al utilizar las dos variables que pasa con el ajuste de la regresión?. Como es la interpretación de los coeficientes?

## Comparando los modelos

Un modelo perfecto se vería como una línea recta a 45 grados como la que vemos en gris. Ya veremos cómo cuantificar la bondad de ajuste pronto.

### Ejercicio


<div id="caja3" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/en_accion.png" style="align:left"/> </div>
  <div style="float:left;width: 85%;"><label>Ajustar el modelo usando TODAS las variables, usando X = df. Esto mejora el ajuste? (comparar el EMC).</label></div>
</div>


In [ ]:
lm = linear_model.LinearRegression()

#X = df[['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT']]

#Otra opción para definir X
X = df[list(df.columns.values)]

y = targets["MEDV"]

model = lm.fit(X, y)
predictions = model.predict(X)

# Graficamos el modelo
plt.plot(y,y, '-.',c='grey')
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicciones de MEDV usando todas las variables")
plt.ylabel("Valores reales MEDV")
plt.show()
print ("EMC:", mean_squared_error(y, predictions))
print ("¿Mejora?: ", mean_squared_error(y, predictions) < prevMSE)

print (sum_mod(model, X))

<div class="alert alert-block alert-success">
<b>Conclusiones:</b> 
    En la primera aproximación evaluamos variables explicativas como RM y LSTAT. De cada variable relevada el modelo de regresión lineal muestra un R cuadrado (R2) de 0.48 y 0.54 respectivamente. Al combinar ambas en un modelo de regresion multiple el R2 aumenta a 0.63 con un EMC de 30.   
    
Por último ejercicio incluye agregar todas las variables del dataset para generar y ajustar  a un modelo multiple con 13 variables explicativas. En consecuencia obtenemos un modelo con un R2 de 0.74 y un EMC de 21   
    
</div>

## Introducción a Statsmodels

### Statsmodels vs sklearn. 
Si ya estamos familiarizados con scikit-learn porque usar otra libreria?. Como vimos en clase hay muchos supuestos detrás de un simple modelo regresión lineal. Por ende si deseamos un análisis estadístico completo  de los p-values de los coeficientes, desvío estándar, errores de cada parámetro, test-t o algún otro, necesitaremos mudarnos de librería.  


Veamos ahora cómo se usa otra librería popular para realizar regresiones como Statsmodels.

In [ ]:
# Importamos la api.
import statsmodels.api as sm

# De manera análoga a la vista en el primer ejercicio, definimos el vector de variables con la primer variable RM.
X = df[["RM"]]
y = targets[["MEDV"]]

# Tenemos que agregar explícitamente a una constante:
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
predictions = model.predict(X)

# Graficamos los resultados
plt.plot(y,y, '-.', c='grey')
plt.scatter(predictions, y, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicciones usando RM")
plt.ylabel("Valores reales MEDV")
plt.show()

# Imprimimos el MSE y un resumen del modelo
print ("EMC:", mean_squared_error(y, predictions))
print (model.summary())

Que podemos decir del p-value para la variable RM. 


* ¿Cómo fue la performance de esta libreria (comparar ECM con sklearn)? 
* ¿Que puede decirse del coeficiente de RM?¿Es significativo?¿Que significa esto?
* ¿ Que es el 'R ajustado'? ¿Para que sirve?

## Referencias


<div id="caja8" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/para_saber_mas.png" style="align:left"/> </div>
  <div style="float:left;width: 85%;"><label></label></div>
</div>


Recursos extra: 
* Si queres entender de manera interactiva las regresiones lineales no dejes de ver este [recurso](http://setosa.io/ev/ordinary-least-squares-regression).

* Repaso de conceptos: [R ajustado:](https://www.statisticshowto.datasciencecentral.com/adjusted-r2/)